In [ ]:
# only keep points in the focus states
focus_states_id_dict = {
    20: "jharkhand",
    22: "chhattisgarh",
    8: "rajasthan",
    23: "madhya pradesh",
    18: "assam",
    16: "tripura",
}

focus_states_filter = request_points_gdf["pc11_s_id"].isin(focus_states_id_dict.keys())
points_gdf_focus = request_points_gdf[focus_states_filter]
points_gdf_focus.shape